<a href="https://colab.research.google.com/github/polaris2010/Python-edication/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22jinja2%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Шаблонизатор здорового человека. Jinja

Это сторонняя библиотека - нужно явно установить:

In [ ]:
!pip install Jinja2

В colab не нужно - стоит по умолчанию:

In [ ]:
import jinja2

## Hello world! с Jinja

In [ ]:
import jinja2
environment = jinja2.Environment()
template = environment.from_string("Hello, {{ name }}!")

template.render(name="World")

'Hello, World!'

Основной компонент Jinja - класс Environment(). В этом примере вы создаете среду Jinja без каких-либо аргументов, но это не надолго :) Позже вы измените параметры среды, чтобы настроить свою среду.

Здесь вы создаете простую среду, в которую загружаете строку `Hello, {{ name }}!` в качестве шаблона.

Данный пример несильно отличается от f-строк, но здесь показаны основные шаги при работе с Jinja:

- **Загрузка шаблона.** Загружаем источник, содержащий переменные-филлеры (заполнители). По умолчанию они заключены в пару фигурных скобок ({{ }}).
- **Рендеринг (отображение) шаблона.** Заполняем филлеры содержимым, например,переданными аргументами.

## Внешний файл в качестве шаблона

Создадим jinja-шаблон в виде отдельного файла:

In [ ]:
%%file message.txt
Hello {{ name }}!

I'm happy to inform you that you did very well on today's {{ test_name }}.
You reached {{ score }} out of {{ max_score }} points.

See you tomorrow!
Anke

Writing message.txt


Теперь напишем программу, которая составит на основе этого шаблона письмо каждому из студентов:

In [ ]:
from jinja2 import Environment, FileSystemLoader

# Информация о студентах
max_score = 100
test_name = "Python Challenge"
students = [
    {"name": "Sandrine",  "score": 100},
    {"name": "Gergeley", "score": 87},
    {"name": "Frieda", "score": 92},
]

# Создаем объект-среду для Jinja
environment = Environment(loader=FileSystemLoader("./"))

# Загружаем шаблон из файла
template = environment.get_template("message.txt")

# Для каждого студента
for student in students:
    # Формируем имя файла письма
    filename = f"message_{student['name'].lower()}.txt"
    # Рендерим шаблон, заполняя поля
    content = template.render(
        student,
        max_score=max_score,
        test_name=test_name
    )
    # Сохраняем письма
    with open(filename, mode="w", encoding="utf-8") as message:
        message.write(content)
        print(f"... wrote {filename}")

... wrote message_sandrine.txt
... wrote message_gergeley.txt
... wrote message_frieda.txt


Проверим одно из писем:

In [ ]:
!cat message_gergeley.txt

## Условный оператор

Пока что наши шаблоны просто позволяют подставить вместо пропуска какое-либо значение, но Jinja может больше - например, добавить вариантивность и цикличность в наши шаблоны.

Доработаем предыдущий пример, добавив в него не самых успешных студентов:

In [ ]:
students = [
    {"name": "Sandrine",  "score": 100},
    {"name": "Gergeley", "score": 87},
    {"name": "Frieda", "score": 92},
    # Не самые успешные
    {"name": "Fritz", "score": 40},
    {"name": "Sirius", "score": 75},
]

Для этих товарищей будет неуместно писать "happy to inform you that you did very well", лучше написать "sorry to inform you that you did not do so well". Поэтому добавим в наш шаблон условный оператор:

In [ ]:
%%file message.txt
Hello {{ name }}!

{% if score > 80 %}
I'm happy to inform you that you did very well on today's {{ test_name }}.
{% else %}
I'm sorry to inform you that you did not do so well on today's {{ test_name }}.
{% endif %}
You reached {{ score }} out of {{ max_score }} points.

See you tomorrow!
Anke

Overwriting message.txt


Помимо переменных (`{{ }}`), можем обозначать и блоки строк внутри шаблона с помощью `{% %}` и спец. слова внутри - в случае с условным оператором это `{% if condition %}`, `{% else %}` и `{% endif %}` (может быть еще `{% elif %}`).

In [ ]:
environment = Environment(loader=FileSystemLoader("./"))
template = environment.get_template("message.txt")

for student in students:
    filename = f"message_{student['name'].lower()}.txt"
    content = template.render(
        student,
        max_score=max_score,
        test_name=test_name
    )
    with open(filename, mode="w", encoding="utf-8") as message:
        message.write(content)
        print(f"... wrote {filename}")

... wrote message_sandrine.txt
... wrote message_gergeley.txt
... wrote message_frieda.txt
... wrote message_fritz.txt
... wrote message_sirius.txt


Посмотрим на сообщение для одного из неуспешных на тесте студентов:

In [ ]:
!cat message_sirius.txt

Hello Sirius!


I'm sorry to inform you that you did not do so well on today's Python Challenge.

You reached 75 out of 100 points.

See you tomorrow!
Anke

In [ ]:
!cat message_frieda.txt

Hello Frieda!


I'm happy to inform you that you did very well on today's Python Challenge.

You reached 92 out of 100 points.

See you tomorrow!
Anke

## Цикл в Jinja

Так же структурой выходного файла можно управлять с помощью цикла `for`. Для примера - создадим html-файл с результатами всех студентов:

In [ ]:
%%file results.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Results</title>
</head>

<body>
  <h1>{{ test_name }} Results</h1>
  <ul>
  {% for student in students %}
    <li>
      <em>{{ student.name }}:</em> {{ student.score }}/{{ max_score }}
    </li>
  {% endfor %}
  </ul>
</body>
</html>

Writing results.html


В этом примере jinja пройдется по списку `students` и воспользуется значениями словарей в нем для заполнения полей.

Аналогично условному оператору, для цикла нам нужны `{% for %}` и `{% endfor %}`.

Можем так же добавить в этот пример и условный оператор:

In [ ]:
%%file results.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Results</title>
</head>

<body>
  <h1>{{ test_name }} Results</h1>
  <ul>
  {% for student in students %}
    <li>
      {% if student.score > 80 %}😃{% else %}☹️{% endif %}
      <em>{{ student.name }}:</em> {{ student.score }}/{{ max_score }}
    </li>
  {% endfor %}
  </ul>
</body>
</html>

Overwriting results.html


В итоге получим следующий код для файла:

In [ ]:
environment = Environment(loader=FileSystemLoader("./"))
results_filename = "students_results.html"
results_template = environment.get_template("results.html")
context = {
    "students": students,
    "test_name": test_name,
    "max_score": max_score,
}
with open(results_filename, mode="w", encoding="utf-8") as results:
    result_rendered_html = results_template.render(context)
    results.write(result_rendered_html)
    print(f"... wrote {results_filename}")

... wrote students_results.html


Посмотрим на результат с помощью инструментов `colab`:

In [ ]:
import IPython
IPython.display.HTML(filename='./students_results.html')

## Вложенность шаблонов

Часто при создании своего приложения приходится создавать несколько шаблонов (использовать if становится недостаточно). При этом часто разные шаблоны чаще всего все равно содержат много схожих элементов. Чтобы избежать дублирования текста в шаблонах, в Jinja имеется механизм вложенности шаблонов.

Чаще всего выделяют базовый шаблон и дочерние шаблоны. Вложенность (или наследуемость) осуществляется с помощью "переопределения" отдельных именованых блоков текста в шаблоне, обрамленных с помощью `{% block block_name %}` и `{% endblock %}`.

Рассмотрим базовый шаблон:

In [ ]:
%%file base.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>{% block title %}{{ title }}{% endblock title %}</title>
</head>

<body>
  {% block content %}
    <h1>Welcome to {{ title }}!</h1>
  {% endblock content %}
</body>
</html>

Writing base.html


В нем мы выделили два блока, которые сможем переписать - `title` и `content`. Содержимое базового шаблона внутри `{% block block_name %}` и `{% endblock %}` - заглушка на тот случай, если в дочернем шаблоне мы этот блок не переопределим.

Теперь создадим дочерний шаблон с результатами теста:

In [ ]:
%%file results.html
{% extends "base.html" %}

{% block content %}
<h1>{{ test_name }} {{ title }}</h1>
<ul>
{% for student in students %}
  <li>
    {% if student.score > 80 %}😃{% else %}☹️{% endif %}
    <em>{{ student.name }}:</em> {{ student.score }}/{{ max_score }}
  </li>
{% endfor %}
</ul>
{% endblock content %}

Overwriting results.html


Связь между дочерних шаблонов с родительским осуществляется с помощью `{% extends "parent_template" %}`.

Переопределение блоков - с помощью создания блоков в дочернем шаблоне.

In [ ]:
environment = Environment(loader=FileSystemLoader("./"))
results_filename = "students_results.html"
results_template = environment.get_template("results.html")
context = {
    "students": students,
    "test_name": test_name,
    "max_score": max_score,
}
with open(results_filename, mode="w", encoding="utf-8") as results:
    result_rendered_html = results_template.render(context)
    results.write(result_rendered_html)
    print(f"... wrote {results_filename}")

... wrote students_results.html


In [ ]:
import IPython
IPython.display.HTML(filename='./students_results.html')